In [29]:
# MLP

# função de ativação

funcao.ativacao <- function(v) {
        y <- 1 / (1 + exp(-v))
        return (y)
}

# derivada da função de ativação

der.funcao.ativacao <- function(y) {
        derivada <- y * (1 - y)
        return (derivada)
}

# arquitetura
arquitetura <- function(num.entrada, num.escondida, num.saida,
                            funcao.ativacao, der.funcao.ativacao) {
    arq <- list()
    arq$num.entrada <- num.entrada
    arq$num.escondida <- num.escondida
    arq$num.saida <- num.saida
    arq$funcao.ativacao <- funcao.ativacao
    arq$der.funcao.ativacao <- der.funcao.ativacao
    
    # 2 neurônios + bias
    #      Ent1    Ent2     Bias
    # 1    w11     w12      w13 
    # 2    w21     w22      w23
    
    # pesos conectando a camada de entrada com a escondida
    num.pesos.entrada.escondida <- (num.entrada+1) * num.escondida
    arq$escondida <- matrix(runif(min=-0.5, max=0.5, num.pesos.entrada.escondida),
                            nrow=num.escondida, ncol=num.entrada+1)
    
    # pesos conectando a camada escondida com a saída
    num.pesos.escondida.saida <- (num.escondida+1) * num.saida
    arq$saida <- matrix(runif(min=-0.5, max=0.5, num.pesos.escondida.saida),
                            nrow=num.saida, ncol=num.escondida+1)
    
    return(arq)
}

In [32]:
# Propagação
mlp.propagacao <- function(arq, exemplo) {
    # Entrada -> Escondida
    v.entrada.escondida <- arq$escondida %*% as.numeric(c(exemplo,1))
    y.entrada.escondida <- arq$funcao.ativacao(v.entrada.escondida)
    
    
    # Escondida -> Saída
    v.escondida.saida <- arq$saida %*% c(y.entrada.escondida,1)
    y.escondida.saida <- arq$funcao.ativacao(v.escondida.saida)
    
    # Resultados
    resultados <- list()
    resultados$v.entrada.escondida <- v.entrada.escondida
    resultados$y.entrada.escondida <- y.entrada.escondida
    resultados$v.escondida.saida <- v.escondida.saida
    resultados$y.escondida.saida <- y.escondida.saida
    
    return(resultados)
}

In [34]:
mlp.propagacao(arq, dados[4, 1:2])

0.3852131
1.0278886
0.5951298
0.7365063
-0.09191774
0.4770367


In [51]:
# Fase de backpropagation
mpl.backpropagation <- function(arq, dados, n, limiar) {
    erroQuadratico <- 2*limiar
    epocas <- 0
    
    # Treina eqto erroQuadratico > limiar
    while (erroQuadratico > limiar) {
        erroQuadratico <- 0
        
        # Treino para todos os exemplos
        for (i in 1:nrow(dados)) {
            # exemplo de entrada
            x.entrada <- dados[i, 1:arq$num.entrada]
            x.saida <- dados[i, ncol(dados)]
            #x.saida <- dados[i, -1]
            
            # saida da rede para o exemplo
            resultado <- mlp.propagacao(arq, x.entrada)
            y <- resultado$y.escondida.saida
            
            # Erro
            erro <- x.saida - y
            erroQuadratico <- erroQuadratico + erro*erro
            
            # gradiente local do neurônio de saída
            # erro * derivada da funcao de ativacao
            grad.local.saida <- erro * arq$der.funcao.ativacao(y)
            
            
            # gradiente local dos neurônios escondidos
            # derivada da funcao de ativação * (sum gradientes locais * pesos)
            pesos.saida <- arq$saida[,1:arq$num.escondida]
            grad.local.escondida <- as.numeric(arq$der.funcao.ativacao(resultado$y.entrada.escondida)) *
                                         (grad.local.saida %*% pesos.saida)
            
            # ajuste dos pesos
            # saida
            arq$saida <- arq$saida + n * (grad.local.saida %*% c(resultado$y.entrada.escondida, 1))
            # Escondida
            arq$escondida <- arq$escondida + n * (t(grad.local.escondida) %*% as.numeric(c(x.entrada, 1)))
        }
        
        erroQuadratico <- erroQuadratico/nrow(dados)
        cat("Erro quadrático médio = ", erroQuadratico, "\n")
        epocas <- epocas + 1
    }
    
    retorno <- list()
    retorno$arq <- arq
    retorno$epocas <- epocas
    
    return(retorno)
}

In [2]:
arq <- arquitetura(2, 2, 1, funcao.ativacao, der.funcao.ativacao)
#arq
dados <- read.table('dataset/XOR.txt')
#dados

modelo <- mpl.backpropagation(arq, dados, 30, 1e-3)

V1,V2,V3
0,0,0
0,1,1
1,0,1
1,1,0


In [63]:
mlp.propagacao(modelo$arq, c(0, 1))$y.escondida.saida
modelo$epocas

0.9927911


[1] 415